# Limpando os dados da nossa base

### Importando a base de clientes de um banco com o resultado do pagamento ou não de um empréstimo

In [1]:
import pandas as pd

In [2]:
base = pd.read_excel("ChavesClientes.xlsx",sheet_name = "base")

In [3]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento
0,1,32FC,Ccinza,Basic-Alpha,1
1,2,25MV,AAmarelo,Black,1
2,3,27MV,B-Amarelo,Basic-Beta,1
3,4,26FD,BPreto,Black,0
4,5,26FD,C-Amarelo,Black,0


**Verificando a cardinalidade desses dados**

In [4]:
base.groupby(["Pagamento","ChaveSituacao"])["Pagamento"].count()

Pagamento  ChaveSituacao
0          26FD             2
           28FC             2
           28MD             1
           30FC             1
           31MD             1
1          25FD             1
           25FV             1
           25MV             1
           26MC             2
           27MC             1
           27MD             2
           27MV             1
           28FS             1
           29MV             1
           31MV             1
           32FC             1
Name: Pagamento, dtype: int64

### Ao analisar essa base, percebemos que existe uma alta cardinalidade e por isso precisamos tratar esses dados

- Ao mapear com o responsável pela área, ele informou o que representa cada coluna:
    - ChaveSituacao: formado por:
        - Idade do cliente (a idade mínima para ser cliente é 18 anos)
        - Gênero do cliente:
            - M: Masculino
            - F: Feminino
        - Estado civil do cliente:
            - S: solteiro
            - C: casado
            - D: divorciado
            - V: viúvo
    - ClassRisco: formado por:
        - Classificação do cliente como (A,B,C) e indicador (+,- ou vazio)
        - Cor do cliente de acordo com um modelo de churn interno da empresa
    - CatCliente: formado por:
        - Categoria do cartão: qual o tipo de cartão do cliente:
            - Basic
            - Black
            - Platinum
        - Categoria VIP: categoria do cliente VIP (caso exista)
            - Alpha
            - Beta

**Podemos começar tratando simplesmente separando a string da coluna**

In [5]:
texto = '32FC'

In [9]:
texto[2:3]

'F'

In [14]:
base['Idade'] = base.ChaveSituacao.str[:2]
base['Genero'] = base.ChaveSituacao.str[2:3]
base['EstadoCivil'] = base.ChaveSituacao.str[-1]

In [19]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C
1,2,25MV,AAmarelo,Black,1,25,M,V
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V
3,4,26FD,BPreto,Black,0,26,F,D
4,5,26FD,C-Amarelo,Black,0,26,F,D


In [16]:
base.groupby(["Pagamento","EstadoCivil"])["Pagamento"].count()

Pagamento  EstadoCivil
0          C              3
           D              4
1          C              4
           D              3
           S              1
           V              5
Name: Pagamento, dtype: int64

**Podemos também fazer o split (divisão) de um valor baseado em um delimitador**

In [28]:
texto = 'Basic-Alpha'

In [29]:
texto.split('-') 

['Basic', 'Alpha']

In [35]:
base['Categoria'] = base.CatCliente.str.split('-').str.get(0)
base['CatVIP'] = base.CatCliente.str.split('-').str.get(1)

In [36]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,Categoria,CatVIP
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta
3,4,26FD,BPreto,Black,0,26,F,D,Black,NaN
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN


**Caso a gente não consiga separar pela posição do texto ou por um delimitador, ainda temos a opção de usar o Regex**

In [37]:
import re

In [41]:
re.findall('Lucas','meu nome é Lucas')

[]

- Regex:
    - ^: Começa com
    - $: Termina com
    - *: O último caracter repetido 0 ou mais vezes
    - +: O último caracter repetido 1 ou mais vezes
    - ?: O último caracter repetido 0 ou 1 vez
    - [A-Z]: qualquer valor em maiúsculo

In [47]:
# Buscando Lucas no texto abaixo
re.findall('Lu*cas','meu nome é Lcas')

['Lcas']

In [49]:
# Verificando se começa ou termina com um caracter (usando ˆ, $)
re.findall('^Lucas','meu nome é Lucas')

[]

In [53]:
# Verificando se existe parte da palavra Lucas (usando *, +, ?)
re.findall('Lu+cas','meu nome é Lucas')

['Lucas']

In [66]:
# Buscando por letras maiúsculas
re.findall('[A-Z]+\w{4}','meu nome é Lucas')

['Lucas']

In [74]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,Categoria,CatVIP
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta
3,4,26FD,BPreto,Black,0,26,F,D,Black,NaN
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN


In [84]:
# Buscando agora a classificação do cliente dentro da coluna "ClassRisco"
re.findall('^[A-Z][^A-Za-z]?','AAmarelo')

['A']

In [88]:
base['Risco'] = base.ClassRisco.apply(lambda x: re.findall('^[A-Z][^A-Za-z]?',x)[0])

In [89]:
base.head()

,ID,ChaveSituacao,ClassRisco,CatCliente,Pagamento,Idade,Genero,EstadoCivil,Categoria,CatVIP,Risco
0,1,32FC,Ccinza,Basic-Alpha,1,32,F,C,Basic,Alpha,C
1,2,25MV,AAmarelo,Black,1,25,M,V,Black,NaN,A
2,3,27MV,B-Amarelo,Basic-Beta,1,27,M,V,Basic,Beta,B-
3,4,26FD,BPreto,Black,0,26,F,D,Black,NaN,B
4,5,26FD,C-Amarelo,Black,0,26,F,D,Black,NaN,C-


In [90]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     object
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   CatVIP         8 non-null      object
 10  Risco          20 non-null     object
dtypes: int64(2), object(9)
memory usage: 1.8+ KB


**Transformando a coluna 'Idade' em numérico**

In [91]:
base['Idade'] = pd.to_numeric(base['Idade'])

In [92]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     int64 
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   CatVIP         8 non-null      object
 10  Risco          20 non-null     object
dtypes: int64(3), object(8)
memory usage: 1.8+ KB


**Por fim, tratando os valores vazios**

In [95]:
base.loc[base.CatVIP.isnull(),"CatVIP"]="Comum"

In [96]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             20 non-null     int64 
 1   ChaveSituacao  20 non-null     object
 2   ClassRisco     20 non-null     object
 3   CatCliente     20 non-null     object
 4   Pagamento      20 non-null     int64 
 5   Idade          20 non-null     int64 
 6   Genero         20 non-null     object
 7   EstadoCivil    20 non-null     object
 8   Categoria      20 non-null     object
 9   CatVIP         20 non-null     object
 10  Risco          20 non-null     object
dtypes: int64(3), object(8)
memory usage: 1.8+ KB


**Agora podemos criar uma nova base apenas com as colunas que acabamos de criar**

In [99]:
base.groupby(["Pagamento","Risco"])["Pagamento"].count()

Pagamento  Risco
0          B        2
           C-       5
1          A        3
           A-       4
           B-       2
           C        3
           C+       1
Name: Pagamento, dtype: int64